## Farhood Ensan, Areeb Syed, Ziqian Cui, Bernard Wong

In [1]:
# Importing required libraries
import shapely
import geopandas as gpd
import pandas as pd
import numpy as np
import random
import matplotlib as plt
import plotly
import plotly.express as px
import plotly.graph_objects as go
import os.path
import json

from mesa_geo import GeoAgent, GeoSpace
from mesa.time import BaseScheduler
from mesa.time import SimultaneousActivation
from mesa import datacollection
from mesa.datacollection import DataCollector
from mesa import Model
from shapely.geometry import Polygon, Point, LineString
from PIL import Image, ImageDraw, ImageFont

# silencing all warnings
import warnings
warnings.filterwarnings('ignore')

In [2]:
#!pip install psutil

In [3]:
# importing methods from other .py files
import os
import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)
    
from src.data.build_datasets import DatasetMaker
from src.models.run_model import RunAll
from src.visualization.viz import Visualize
from src.visualization.viz_gif import GifMaker


In [4]:
#changing the working directory to the main folder (RUN ONLY ONCE)
os.chdir('../')
os.getcwd()

'/Users/ifarhood/Documents/GitHub/Bus_Simulation_Model'

In [5]:
routes = pd.read_csv('test/testdata/routes_v1.csv')#,index=True)
routes.index = routes.iloc[:,0]
routes = routes.drop(['Unnamed: 0'],axis=1)
routes.loc['Route:T017  BA ']

start_loc              ('PARADISE VALLEY RD@PARKWOOD DR', '1290H5')
stop_locs         [('PARADISE VALLEY RD@PARKWOOD DR', '1290H5'),...
dest_loc          ('BARNARD ASIAN PACIFIC LANG ACADEMY, LZ', 'OR...
start_time                                                 06:31:00
stop_times        [datetime.time(6, 31), datetime.time(6, 32), d...
dest_time                                                  07:30:00
kids_pickup                                             [3, 12, 11]
kids_dropoff                                                   [26]
daysofweek                                ['M', 'T', 'W', 'H', 'F']
morning                                                        True
stop_geos         [<shapely.geometry.point.Point object at 0x11c...
stop_dists        [456.8185991001796, 7281.714734990269, 3107.76...
stop_durations                              [1.0, 14.0, 10.0, 34.0]
speeds_by_stop    [17.03127738 19.39139577 11.58647572 15.15637213]
outlier                                         

In [6]:
routes.loc['Route:T017  BA ']['stop_durations']

'[1.0, 14.0, 10.0, 34.0]'

In [7]:
routes.loc['Route:T017  BA ']['kids_pickup']# + routes.loc['Route:T017  BA ']['kids_dropoff']

'[3, 12, 11]'

In [8]:
# True: Uses Given Parameters in config/data_params.json (could be changed by user)
# False: Uses test Parameters in test/testdata/testdata.json (To simulate for the replication study)

use_given_params = True

In [9]:
def params_setter(route, params):
    dur = routes.loc[route]['stop_durations']
    dur = eval(dur)
    kids = routes.loc[route]['kids_pickup']
    kids = eval(kids)
    
    if len(dur)>len(kids):
        dur = dur[1:]
    
    kids = list(np.cumsum(np.array(kids)))
    dur = list(np.cumsum(np.array(dur)))
    #print(kids)
    params["bus_stop_student_count"] = kids
    params["bus_stop_minutes_count"] = [0]+dur[:-1]
    params["student_num"] = kids[-1]
    params["steps"] = dur[-1]
    return params

1) zigzag seating 
>> 15 day simulation
aerosol_new.py >> aerosol

2) implement aerosol model (take in the values) 
    check the input parameters needed (volume of the bus, # of students, air refresh rate)
    masks, etc
3) update droplet model
4) get one bus for the demo
    accurate demo for that bus based on dimentions
        randomize bus selection based on total number of students and wheelchair
5) 

In [10]:
params_setter('Route:T017  BA ', {})

{'bus_stop_student_count': [3, 15, 26],
 'bus_stop_minutes_count': [0, 14.0, 24.0],
 'student_num': 26,
 'steps': 58.0}

In [11]:
# using given parameters in config folder
if use_given_params == True:
    print('> using given parameters')
    # extracting parameters from config file
    params = DatasetMaker().make_data('bus_params')
    
else:
# using test parameters to simulate the replication paper
    try:
        print('> using test parameters')
        # extracting parameters from test file
        params = DatasetMaker().make_data('test')

    except:
        print('invalid data')
        
        
params_setter('Route:T017  BA ', params)        
# initiating the model, running the model, and recording agent data for each step
agent_data, results = RunAll().run_program(params)

print('> Working on Visualization \n .')
# making the results graph showing number of healthy and sick passangers in each step
Visualize().make_viz(results,params)
print(" .")
#making the gif of the bus showing the location of healhty and sick people in each step
GifMaker().make_gif(agent_data,params)
print(" .")
print('> Visualization Done')
print('> results saved in figures folder')

> using given parameters
> Working on Visualization 
 .
 .
 .
> Visualization Done
> results saved in figures folder
